In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import yfinance as yfin
import utils as erk 
import FactorModelLibForMOOC as fm 
import config_demo_final as config 
%reload_ext autoreload
%autoreload 2

In [4]:
tickers = ['aapl', 'xom', 'bac', 'wmt', 'avgo', 'lly', 'ge', 'pg', 't', 'ups']


dataframe = yfin.download(tickers, period = 'max')['Adj Close']
returns = dataframe.pct_change().dropna()

mreturns = returns.resample('M').apply(erk.compound).to_period('M')

mreturns 


[*********************100%%**********************]  10 of 10 completed
C:\Users\Admin\AppData\Local\Temp\ipykernel_7692\3115196414.py:7: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  mreturns = returns.resample('M').apply(erk.compound).to_period('M')


Ticker,AAPL,AVGO,BAC,GE,LLY,PG,T,UPS,WMT,XOM
Date,,,,,,,,,,
2009-08,0.026234,0.124846,0.053294,-0.028651,-0.013535,0.051496,0.019570,0.012767,0.044284,-0.002261
2009-09,0.101897,-0.062088,-0.037505,0.188284,-0.012851,0.070412,0.036852,0.056304,-0.034991,-0.007809
2009-10,0.016995,-0.121266,-0.138298,-0.131547,0.029670,0.009019,-0.030221,-0.049407,0.012019,0.044600
2009-11,0.060531,0.057333,0.087106,0.123422,0.095170,0.075000,0.049474,0.078965,0.098028,0.053536
2009-12,0.054124,0.153216,-0.049244,-0.049415,-0.027770,-0.027587,0.040460,-0.001740,-0.015224,-0.091647
...,...,...,...,...,...,...,...,...,...,...
2024-04,-0.006706,-0.018968,-0.023998,0.157188,0.004036,0.012350,-0.024622,-0.007737,-0.013628,0.017464
2024-05,0.130222,0.021749,0.080519,0.020517,0.052027,0.008211,0.078745,-0.047488,0.111839,-0.000487
2024-06,0.095553,0.212320,0.000547,-0.037362,0.103664,0.002309,0.048847,-0.014971,0.029653,-0.018250


In [35]:
all_data = pd.read_csv(config.dataPath)
all_data[config.dateName] = pd.to_datetime(all_data[config.dateName])
startdate = '2009-08-01'
enddate = '2019-08-01'
all_data


C:\Users\Admin\AppData\Local\Temp\ipykernel_7692\3466630119.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data[config.dateName] = pd.to_datetime(all_data[config.dateName])


,Date,World Equities,10-year US Treasuries,High Yield,Inflation Protection,Currency Protection,U.S. Equity,SP500 Total Return,S&P 500,International Equity,U.S. Treasury 20 years,Corporate Bond,Real Estate,Commodity,TIPS
0,1995-01-01,-0.020349,0.022922,0.032048,0.006524,-0.003404,0.022239,0.025954,0.025897,-0.039228,0.025987,0.017042,-0.014847,-0.023908,0.029446
1,1995-02-01,0.010680,0.030014,0.013531,-0.038375,-0.020587,0.036258,0.035540,0.038974,-0.000434,0.026960,0.029870,0.032460,0.002221,-0.008361
2,1995-03-01,0.045804,0.007195,0.025871,-0.023184,-0.042060,0.030791,0.035454,0.029502,0.061949,0.009477,0.011937,-0.005961,0.019558,-0.015989
3,1995-04-01,0.036372,0.016973,0.030836,-0.003600,-0.015827,0.022403,0.026256,0.029414,0.037154,0.017695,0.010995,0.002259,0.038085,0.013373
4,1995-05-01,0.011040,0.054900,0.006743,-0.054282,0.009121,0.036521,0.040894,0.039949,-0.009769,0.076715,0.053217,0.045025,-0.009417,0.000618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,2016-02-01,NaN,NaN,0.044208,NaN,-0.023402,0.023712,0.022927,NaN,-0.011061,0.028198,0.006360,0.021567,0.009421,NaN
254,2016-03-01,NaN,NaN,0.039953,NaN,-0.037563,0.051944,0.049537,NaN,0.036116,0.001023,0.030894,0.072566,0.022858,NaN
255,2016-04-01,NaN,NaN,0.007221,NaN,-0.020016,0.008720,0.005354,NaN,0.051411,NaN,0.011063,-0.002171,0.109346,NaN
256,2016-05-01,NaN,NaN,0.010833,NaN,0.033049,0.011779,0.011375,NaN,-0.010807,NaN,0.001398,0.009775,0.039373,NaN


In [49]:
expvar = all_data[ (all_data['Date'] <= enddate) & (all_data['Date'] >= startdate) ].copy()
expvar = expvar[['Date']+ config.factorName]
expvar

,Date,World Equities,10-year US Treasuries,High Yield,Inflation Protection,Currency Protection
175,2009-08-01,-0.000009,0.010147,0.059750,-0.001367,0.000849
176,2009-09-01,0.041168,0.010324,0.017865,0.010728,-0.020046
177,2009-10-01,0.006595,-0.003575,0.010129,0.015847,-0.003752
178,2009-11-01,0.060426,0.018977,0.031348,0.008841,-0.019951
179,2009-12-01,0.000832,-0.037937,0.015154,0.016019,0.030559
...,...,...,...,...,...,...
253,2016-02-01,NaN,NaN,0.044208,NaN,-0.023402
254,2016-03-01,NaN,NaN,0.039953,NaN,-0.037563
255,2016-04-01,NaN,NaN,0.007221,NaN,-0.020016
256,2016-05-01,NaN,NaN,0.010833,NaN,0.033049


In [43]:
ff = erk.get_fff_returns()
ff

,Mkt-RF,SMB,HML,RF
1926-07,0.0296,-0.0256,-0.0243,0.0022
1926-08,0.0264,-0.0117,0.0382,0.0025
1926-09,0.0036,-0.0140,0.0013,0.0023
1926-10,-0.0324,-0.0009,0.0070,0.0032
1926-11,0.0253,-0.0010,-0.0051,0.0031
...,...,...,...,...
2024-01,0.0070,-0.0509,-0.0238,0.0047
2024-02,0.0506,-0.0024,-0.0349,0.0042
2024-03,0.0283,-0.0249,0.0420,0.0043
2024-04,-0.0467,-0.0239,-0.0052,0.0047


In [44]:
# mkt_excess = mreturns["2009-08":"2014-01"] - ff.loc["2009-08":"2014-01", ['RF']].values
mkt_excess = mreturns.loc[ startdate : enddate , ['GE']]
mkt_excess 


Ticker,GE
Date,
2009-08,-0.028651
2009-09,0.188284
2009-10,-0.131547
2009-11,0.123422
2009-12,-0.049415
...,...
2019-04,0.018018
2019-05,-0.071780
2019-06,0.113359


In [7]:
mkt_rf = ff.loc["2009-08":"2014-01", ['Mkt-RF', 'SMB', 'HML']]
mkt_rf

,Mkt-RF,SMB,HML
2009-08,0.0333,-0.0108,0.0763
2009-09,0.0408,0.0243,0.0105
2009-10,-0.0259,-0.0435,-0.0421
2009-11,0.0556,-0.0240,-0.0034
2009-12,0.0275,0.0605,-0.0017
2010-01,-0.0336,0.0040,0.0043
2010-02,0.0340,0.0119,0.0323
2010-03,0.0631,0.0148,0.0221
2010-04,0.0200,0.0487,0.0289
2010-05,-0.0789,0.0009,-0.0244


In [45]:
train = mkt_excess.copy()


In [46]:
import statsmodels as sm
y = mkt_excess.copy()
x = expvar.copy()
x['Intercept'] = 1
r = sm.regression.linear_model.OLS(y,x)
r = r.fit()
r.summary()

MissingDataError: exog contains inf or nans

In [12]:
erk.ff_analysis_series(tickers, mreturns, timeframe_start='2009-08', timeframe_end='2024-05', ff = ff)


,Mkt-RF,SMB,HML,Constant
aapl,1.184317,-0.265332,-0.556087,0.009129
xom,1.002009,0.177795,-0.256787,0.018271
bac,1.335333,0.169719,1.086038,-0.004086
wmt,1.142268,0.214691,0.553673,-0.001841
avgo,0.372569,-0.078286,-0.431207,0.016920
lly,0.498457,-0.737902,0.084345,0.003314
ge,0.552909,-0.338391,0.427611,0.000624
pg,1.023223,-0.157536,0.019894,-0.001949
t,0.545098,-0.563083,-0.184451,0.003645
ups,0.838125,-0.123153,0.897304,-0.000818


In [13]:
options = fm.create_options()
options['name_of_reg'] = 'OLS full data'
options['return_model'] = True
ols_model_train = fm.linear_regression(train, 'GE', config.factorName, options, mkt_rf)

Dependent Variable is GE
Time period is between August 2009 to January 2014 inclusive   
               Intercept    Mkt-RF       SMB       HML
OLS full data  -0.002494  1.343104 -0.160758  0.529972


## Results 

Done FF analysis factor estimate for each of the stock in portfolio, now we can see which stock has which factor tilt. 

Problem 1 --> factor returns may be highly correlated, in which case we need to use 

### Ridge Regression 

Problem 2 --> overfitting when number of factors >> number of assets 

PRoblem 3 --> different factor loading across time period 




In [75]:
ff.corr()

,Mkt-RF,SMB,HML,RF
Mkt-RF,1.000000,0.316064,0.227406,-0.065415
SMB,0.316064,1.000000,0.117448,-0.051639
HML,0.227406,0.117448,1.000000,0.019433
RF,-0.065415,-0.051639,0.019433,1.000000


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

In [15]:
train = mkt_excess.copy()
options = fm.create_options_lasso()
options['lambda_hat'] = .00001
options['print_loadings'] = True
options['name_of_reg'] = 'GE'
options['time_period'] = 'all'
fm.lasso_regression(train, 'GE', config.factorName, options, mkt_rf)

Dependent Variable is GE
Time period is between August 2009 to January 2014 inclusive   
lambda_hat = 1e-05
    Intercept    Mkt-RF       SMB       HML
GE  -0.002446  1.332928 -0.125297  0.510369


## Results

LASSO regression yields a different factor tilted compared to OLS regression

# Cross Validation 

We move on to find a suitable $\lambda$ value 

In [16]:
options = fm.create_options_cv_lasso()
options['name_of_reg'] = 'CV Lasso'
options['max_lambda_hat'] = .001 #This specifies the maximum Alpha value tested by cross validation, minimum value is zero
options['return_model'] = True
options['n_folds'] = 5 #This states the number of folds
lasso_model_train = fm.cross_validated_lasso_regression(train, 'AAPL', config.factorName, options, mkt_rf)

Dependent Variable is AAPL
Time period is between August 2009 to January 2014 inclusive   
Best lambda_hat = 0.00011359913314956731
          Intercept    Mkt-RF  SMB       HML
CV Lasso  -0.002077  1.284289 -0.0  0.302737


# CV Elastic Net 

In [17]:
options = fm.create_options_cv_elastic_net()
options['name_of_reg'] = 'CV Elastic Net'
options['n_folds'] = 5
options['return_model'] = True
el_model_train = fm.cross_validated_elastic_net_regression(train, 'AAPL', config.factorName, options, mkt_rf)

c:\Users\Admin\PycharmProjects\Portfolio Analysis\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.315e-03, tolerance: 2.385e-05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
c:\Users\Admin\PycharmProjects\Portfolio Analysis\.venv\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.616e-04, tolerance: 2.621e-05 Linear regression models with null weight for the l1 regularization term are more efficientl

Dependent Variable is AAPL
Time period is between August 2009 to January 2014 inclusive   
Best lambda_hat = 0.0002310129700083158
Best l1 ratio = 0.15631663157894737
                Intercept    Mkt-RF  SMB       HML
CV Elastic Net   -0.00056  1.175593  0.0  0.387259


In [18]:
# finding fama french 3 factor model monthly returns 

expected = pd.DataFrame(np.array([[0.0434/12, 0.0054/12, -0.0133/12]]), columns= config.factorName)
result  = pd.DataFrame(np.zeros((3, 5)), columns=['Intercept'] + config.factorName + 
                          ['GE Expected Return (Annualized)'])

# using fama french 3 factor model monthly return of May to predict annualised return of AAPL
models = [ols_model_train, lasso_model_train, el_model_train]
for i in range(len(models)): 
    model = models[i]
    result.loc[i,'Intercept'] = model.intercept_
    result.loc[i, config.factorName] = model.coef_
    result.loc[i, 'GE Expected Return (Annualized)'] = 12*model.predict(expected)
result.index = ['OLS', 'LASSO', ' EN CV']

result


,Intercept,Mkt-RF,SMB,HML,GE Expected Return (Annualized)
OLS,-0.002494,1.343104,-0.160758,0.529972,0.020442
LASSO,-0.002077,1.284289,-0.000000,0.302737,0.026783
EN CV,-0.000560,1.175593,0.000000,0.387259,0.039151


In [19]:
from sklearn.metrics import r2_score
test = mreturns["2023-02":"2024-05"] - ff.loc["2023-02":"2024-05", ['RF']].values
test = test.loc["2023-02":"2024-05", ['AAPL']]
test_ff = ff.loc["2023-02":"2024-05", ['Mkt-RF', 'SMB', 'HML']]
predictions = pd.DataFrame(np.zeros((2,1)), columns=['R^2 on Testing Set'])

for i in range(len(models)):

    model = models[i]
    predictions.loc[i, 'R^2 on Testing Set'] = r2_score(test, model.predict(test_ff))

predictions.index = ['OLS', 'LASSO', ' EN CV']


In [170]:
predictions

,R^2 on Testing Set
OLS,-0.367355
LASSO,0.121527
EN CV,0.130506
